In [4]:
import model_loader
import pipeline
from PIL import Image
from pathlib import Path
from transformers import CLIPTokenizer
import torch
import gc

DEVICE = "gpu"

ALLOW_CUDA = True
ALLOW_MPS = True

if torch.cuda.is_available() and ALLOW_CUDA:
    DEVICE = "cuda"
elif (torch.has_mps or torch.backends.mps.is_available()) and ALLOW_MPS:
    DEVICE = "mps"
print(f"Using device: {DEVICE}")

tokenizer = CLIPTokenizer("../data/vocab.json", merges_file="../data/merges.txt")
model_file = "../data/inkpunk-diffusion-v1.ckpt"
models = model_loader.preload_models_from_standard_weights(model_file, DEVICE)

## TEXT TO IMAGE

# prompt = "A dog with sunglasses, wearing comfy hat, looking at camera, highly detailed, ultra sharp, cinematic, 100mm lens, 8k resolution."
prompt = "A cat stretching on the floor, highly detailed, ultra sharp, cinematic, 100mm lens, 8k resolution."
uncond_prompt = ""  # Also known as negative prompt
do_cfg = True
cfg_scale = 8  # min: 1, max: 14

## IMAGE TO IMAGE

input_image = None
# Comment to disable image to image
image_path = "../images/dog.jpeg"
#input_image = Image.open(image_path)


strength = 0.9


## SAMPLER
sampler = "ddpm"
num_inference_steps = 50
seed = 6

# Use torch.no_grad() to avoid memory issues with gradient calculation
with torch.no_grad():
    output_image = pipeline.generate(
        prompt=prompt,
        uncond_prompt=uncond_prompt,
        input_image=input_image,
        strength=strength,
        do_cfg=True,
        cfg_scale=8,
        sampler_name=sampler,
        n_inference_steps=num_inference_steps,
        seed=seed,
        models=models,
        device=DEVICE,
        idle_device="cpu",
        tokenizer=tokenizer,
    )

# Mostrar o guardar la imagen de salida
output_image_pil = Image.fromarray(output_image)
output_image_pil.show()  # Para mostrar la imagen
# output_image_pil.save("output_image.png")  # Para guardar la imagen en disco

# Limpiar memoria de GPU
def free_gpu_memory():
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    elif (torch.has_mps or torch.backends.mps.is_available()):
        torch.mps.empty_cache()

free_gpu_memory()


Using device: cuda


100%|██████████| 45/45 [09:05<00:00, 12.12s/it]
